# Final Project Submission

Please fill out:
* Student name: Austin Towery
* Student pace: self paced
* Scheduled project review date/time: 
* Instructor name: Claude Fried
* Blog post URL:


<font size="5">**Overview**</font>
***
* This analysis focuses on the effect that homeowner renovations have on the sale price and the size of that effect.

<font size="5">**Business Problem**</font>
***
* Real Real Estate Agency is planning on advising clients/homeowners about what effect renovations will have on their home's estimated value


* By analyzing sales data for other homes I will provide guidance on what this effect is and the size of it's impact


* This guidance will allow them to confidently give advice that is backed up by real world data

# Import and Clean Data

In [34]:
# from sklearn.linear_model import LinearRegression
# baseline_model = LinearRegression()

# from sklearn.model_selection import cross_validate, ShuffleSplit
# splitter = ShuffleSplit(n_splits=3, test_size=0.25, random_state=0)



In [35]:
# Import Necessary Libraries
import numpy as np
import pandas as pd

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression

import statsmodels.api as sm
import scipy.stats as stats

import seaborn as sns
import matplotlib as plt
%matplotlib inline


In [36]:
# import data and preview
data = pd.read_csv('data\kc_house_data.csv')
data.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,10/13/2014,221900.0,3,1.00,1180,5650,1.0,NaN,NONE,...,7 Average,1180,0.0,1955,0.0,98178,47.5112,-122.257,1340,5650
1,6414100192,12/9/2014,538000.0,3,2.25,2570,7242,2.0,NO,NONE,...,7 Average,2170,400.0,1951,1991.0,98125,47.7210,-122.319,1690,7639
2,5631500400,2/25/2015,180000.0,2,1.00,770,10000,1.0,NO,NONE,...,6 Low Average,770,0.0,1933,NaN,98028,47.7379,-122.233,2720,8062
3,2487200875,12/9/2014,604000.0,4,3.00,1960,5000,1.0,NO,NONE,...,7 Average,1050,910.0,1965,0.0,98136,47.5208,-122.393,1360,5000
4,1954400510,2/18/2015,510000.0,3,2.00,1680,8080,1.0,NO,NONE,...,8 Good,1680,0.0,1987,0.0,98074,47.6168,-122.045,1800,7503


In [42]:
# create df to work on and view info for data types and null values
df = data
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21597 entries, 0 to 21596
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             21597 non-null  int64  
 1   date           21597 non-null  object 
 2   price          21597 non-null  float64
 3   bedrooms       21597 non-null  int64  
 4   bathrooms      21597 non-null  float64
 5   sqft_living    21597 non-null  int64  
 6   sqft_lot       21597 non-null  int64  
 7   floors         21597 non-null  float64
 8   waterfront     19221 non-null  object 
 9   view           21534 non-null  object 
 10  condition      21597 non-null  object 
 11  grade          21597 non-null  object 
 12  sqft_above     21597 non-null  int64  
 13  sqft_basement  21597 non-null  object 
 14  yr_built       21597 non-null  int64  
 15  yr_renovated   17755 non-null  float64
 16  zipcode        21597 non-null  int64  
 17  lat            21597 non-null  float64
 18  long  

* `'waterfont', 'view', and 'yr_renovated'` are the only columns with obvious missing values


* `'sqft_basement` and `date` are a strings, might need to be converted. Otherwise every other numerical value is an int or float


* There are 4 columns of categorical data

> Since we are looking at data for renovations we will only be able to use entries with a yr_renovated value

In [38]:
# view unique values of yr_renovated column for anything besides null values to remove
df['yr_renovated'].unique()

array([   0., 1991.,   nan, 2002., 2010., 1992., 2013., 1994., 1978.,
       2005., 2003., 1984., 1954., 2014., 2011., 1983., 1945., 1990.,
       1988., 1977., 1981., 1995., 2000., 1999., 1998., 1970., 1989.,
       2004., 1986., 2007., 1987., 2006., 1985., 2001., 1980., 1971.,
       1979., 1997., 1950., 1969., 1948., 2009., 2015., 1974., 2008.,
       1968., 2012., 1963., 1951., 1962., 1953., 1993., 1996., 1955.,
       1982., 1956., 1940., 1976., 1946., 1975., 1964., 1973., 1957.,
       1959., 1960., 1967., 1965., 1934., 1972., 1944., 1958.])

> There are both nan and '0' values that we want to get rid of

In [48]:
# make df with only rows where yr_renovated is not null or 0
df = df[(df['yr_renovated'].notna()) & (df['yr_renovated'] != 0)]
# reprint unique values to make sure we have only what we want
print('***Unique Values***')
print(df['yr_renovated'].unique())
# new lines and dashes for formatting
print('\n')
print('--------------------------------------------------------------------')
print('\n')
# print new df info to see how much is left
print('***New df Info***')
df.info()

***Unique Values***
[1991. 2002. 2010. 1992. 2013. 1994. 1978. 2005. 2003. 1984. 1954. 2014.
 2011. 1983. 1945. 1990. 1988. 1977. 1981. 1995. 2000. 1999. 1998. 1970.
 1989. 2004. 1986. 2007. 1987. 2006. 1985. 2001. 1980. 1971. 1979. 1997.
 1950. 1969. 1948. 2009. 2015. 1974. 2008. 1968. 2012. 1963. 1951. 1962.
 1953. 1993. 1996. 1955. 1982. 1956. 1940. 1976. 1946. 1975. 1964. 1973.
 1957. 1959. 1960. 1967. 1965. 1934. 1972. 1944. 1958.]


--------------------------------------------------------------------


***New df Info***
<class 'pandas.core.frame.DataFrame'>
Int64Index: 744 entries, 1 to 20946
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             744 non-null    int64  
 1   date           744 non-null    object 
 2   price          744 non-null    float64
 3   bedrooms       744 non-null    int64  
 4   bathrooms      744 non-null    float64
 5   sqft_living    744 non-null    int64  
 6   sqft

> That got rid of quite a bit of data unfortunately but now we have the relevant data.

In [57]:
# rename columns for clarity
df.rename(columns={'date': 'date_sold', 'price': 'sale_price'}, inplace=True)
# drop id column since it doesn't effect sale_price
df.drop('id', axis=1, inplace=True)
df.head()

,date_sold,sale_price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
1,12/9/2014,538000.0,3,2.25,2570,7242,2.0,NO,NONE,Average,7 Average,2170,400.0,1951,1991.0,98125,47.7210,-122.319,1690,7639
35,6/13/2014,696000.0,3,2.50,2300,3060,1.5,NO,NONE,Average,8 Good,1510,790.0,1930,2002.0,98115,47.6827,-122.310,1590,3264
95,9/8/2014,905000.0,4,2.50,3300,10250,1.0,NO,NONE,Average,7 Average,2390,910.0,1946,1991.0,98040,47.5873,-122.249,1950,6045
103,10/7/2014,1090000.0,3,2.50,2920,8113,2.0,NO,NONE,Average,8 Good,2920,0.0,1950,2010.0,98004,47.5814,-122.196,2370,8113
125,3/2/2015,1450000.0,4,2.75,2750,17789,1.5,NO,NONE,Average,8 Good,1980,770.0,1914,1992.0,98004,47.6141,-122.212,3060,11275
